# Speech-to-Speech Translation (Google Colab)

This notebook runs a simple **video → speech → translation → TTS → final video** pipeline.

## Steps
1. Install dependencies
2. Upload input video (`input_video.mp4`)
3. Run the pipeline
4. Download `translated_video.mp4`


In [ ]:
# Install dependencies (run once)
!pip -q install moviepy speechrecognition googletrans==4.0.0-rc1 gtts pydub
!apt-get -qq install -y ffmpeg


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langsmith 0.7.1 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is i

In [ ]:
# Upload your input video and rename it to input_video.mp4
from google.colab import files
uploaded = files.upload()


Saving WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4 to WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4


In [ ]:
import moviepy.editor as mp
import speech_recognition as sr
from googletrans import Translator
from gtts import gTTS
import os

def extract_audio(video_path, audio_path='extracted_audio.wav'):
    print(f'Extracting audio from {video_path}...')
    video = mp.VideoFileClip(video_path)
    video.audio.write_audiofile(audio_path)
    return audio_path

def speech_to_text(audio_path):
    print('Converting speech to text...')
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)
    text = recognizer.recognize_google(audio)
    print('Recognized Text:', text)
    return text

def translate_text(text, target_lang='hi'):
    print(f'Translating text to {target_lang}...')
    translator = Translator()
    translated = translator.translate(text, dest=target_lang)
    print('Translated Text:', translated.text)
    return translated.text

def text_to_speech(text, lang='hi', output_audio='translated_audio.mp3'):
    print(f'Generating translated speech in {lang}...')
    # Basic mapping for common full names to codes
    lang_map = {'telugu': 'te', 'hindi': 'hi', 'spanish': 'es', 'french': 'fr', 'tamil': 'ta'}
    lang_code = lang_map.get(lang.lower(), lang)

    tts = gTTS(text=text, lang=lang_code)
    tts.save(output_audio)
    return output_audio

def merge_audio_video(video_path, audio_path, output_video='translated_video.mp4'):
    print('Merging audio with video...')
    video = mp.VideoFileClip(video_path)
    new_audio = mp.AudioFileClip(audio_path)
    if new_audio.duration > video.duration:
        new_audio = new_audio.subclip(0, video.duration)
    final_video = video.set_audio(new_audio)
    final_video.write_videofile(output_video)
    print('Done! Output:', output_video)

def run_pipeline(video_file, target_language='hi'):
    if not os.path.exists(video_file):
        mp4_files = [f for f in os.listdir('.') if f.endswith('.mp4') and f != 'translated_video.mp4']
        if mp4_files:
            video_file = mp4_files[0]
            print(f'Using detected file: {video_file}')
        else:
            print('Error: No video file found. Please upload an mp4 file.')
            return

    audio = extract_audio(video_file)
    text = speech_to_text(audio)
    translated = translate_text(text, target_language)
    tts_audio = text_to_speech(translated, target_language)
    merge_audio_video(video_file, tts_audio)

# Prompt for language selection
target_lang_input = input("Enter the language code (e.g., hi, es, fr, te): ")
run_pipeline('input_video.mp4', target_lang_input)

Enter the language code (e.g., hi, es, fr, te): te
Using detected file: WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4
Extracting audio from WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4...
MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.
Converting speech to text...
Recognized Text: do well in your life and you've achieved every material game that you want which you should please don't give up on that don't become a philosopher before you become rich
Translating text to te...
Translated Text: మీ జీవితంలో బాగా చేయండి మరియు మీరు కోరుకునే ప్రతి మెటీరియల్ గేమ్‌ను మీరు సాధించారు, దయచేసి మీరు దానిని వదులుకోవద్దు, మీరు ధనవంతులు కావడానికి ముందు తత్వవేత్తగా మారకండి
Generating translated speech in te...
Merging audio with video...
Moviepy - Building video translated_video.mp4.
MoviePy - Writing audio in translated_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video translated_video.mp4



t:  98%|█████████▊| 261/265 [00:12<00:00, 23.41it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4, 2764800 bytes wanted but 0 bytes read,at frame 262/265, at time 8.73/8.82 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t: 100%|█████████▉| 264/265 [00:12<00:00, 22.42it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4, 2764800 bytes wanted but 0 bytes read,at frame 263/265, at time 8.77/8.82 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready translated_video.mp4
Done! Output: translated_video.mp4


In [ ]:
# Download the final video
from google.colab import files
files.download('translated_video.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Task
Update the video translation pipeline to support gender selection and a numeric menu-based input system. Modify the `text_to_speech` function to accept a gender parameter and provide a mechanism (like a note or specific voice mapping) for choosing between male and female voices. Implement a numeric menu using `input()` to allow the user to select the gender (1: Men, 2: Women) and the target language (1: Telugu, 2: Hindi, etc.). Finally, refactor the `run_pipeline` call to use these numeric inputs to process the video "WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4" and produce the `translated_video.mp4`.

## Update TTS for Gender Support

### Subtask:
Modify the `text_to_speech` function to accept a gender parameter and handle gender selection logic.


**Reasoning**:
I will update the `text_to_speech` function to include the `gender` parameter and add the requested logic and documentation to support future expansion, as per the instructions.



In [ ]:
def text_to_speech(text, lang='hi', gender='female', output_audio='translated_audio.mp3'):
    """
    Generates translated speech.

    Note: gTTS natively defaults to a female-sounding voice. The 'gender' parameter
    is integrated here to support future expansion with other TTS engines (like pyttsx3
    or Cloud APIs) that support specific voice IDs.
    """
    print(f'Generating translated speech in {lang}...')

    # Acknowledge the selected gender for the pipeline
    gender_map = {'male': '1', 'female': '2'}
    selected_code = gender_map.get(gender.lower(), '2')
    print(f'Selected Gender Logic: {gender} (ID: {selected_code})')

    # Language mapping logic
    lang_map = {'telugu': 'te', 'hindi': 'hi', 'spanish': 'es', 'french': 'fr', 'tamil': 'ta'}
    lang_code = lang_map.get(lang.lower(), lang)

    # Using gTTS for synthesis (defaulting to its standard voice)
    tts = gTTS(text=text, lang=lang_code)
    tts.save(output_audio)
    return output_audio

print('Updated text_to_speech function with gender support.')

Updated text_to_speech function with gender support.


## Create Menu-Based Input

### Subtask:
Implement a numeric menu system using `input()` to capture user preferences for gender and target language.


**Reasoning**:
I will implement a numeric menu-based input system using the `input()` function to capture and map user preferences for gender and target language to their respective internal values ('male'/'female' and language codes).



In [ ]:
def get_user_preferences():
    # 1. Gender Selection Menu
    print("\n--- Select Gender ---")
    print("1: Men")
    print("2: Women")
    gender_choice = input("Enter your choice (1 or 2): ")

    # 2. Language Selection Menu
    print("\n--- Select Target Language ---")
    print("1: Telugu")
    print("2: Hindi")
    print("3: Spanish")
    print("4: French")
    lang_choice = input("Enter your choice (1, 2, 3, or 4): ")

    # 3. Logic to map gender choice
    gender_map = {'1': 'male', '2': 'female'}
    selected_gender = gender_map.get(gender_choice, 'female') # Default to female if invalid

    # 4. Logic to map language choice
    lang_map = {
        '1': {'name': 'Telugu', 'code': 'te'},
        '2': {'name': 'Hindi', 'code': 'hi'},
        '3': {'name': 'Spanish', 'code': 'es'},
        '4': {'name': 'French', 'code': 'fr'}
    }
    lang_info = lang_map.get(lang_choice, {'name': 'Hindi', 'code': 'hi'}) # Default to Hindi
    selected_lang_code = lang_info['code']
    selected_lang_name = lang_info['name']

    # 5. Print resolved selections to verify
    print(f"\nResolved Selections:")
    print(f"Selected Language: {selected_lang_name} ({selected_lang_code})")
    print(f"Selected Gender: {selected_gender}")

    return selected_lang_code, selected_gender

# Execute the menu to capture input
chosen_lang, chosen_gender = get_user_preferences()


--- Select Gender ---
1: Men
2: Women
Enter your choice (1 or 2): 1

--- Select Target Language ---
1: Telugu
2: Hindi
3: Spanish
4: French
Enter your choice (1, 2, 3, or 4): 1

Resolved Selections:
Selected Language: Telugu (te)
Selected Gender: male


## Update Pipeline Integration

### Subtask:
Refactor the `run_pipeline` call and function to incorporate the new gender and language variables captured from the menu system.


**Reasoning**:
I will update the run_pipeline function to accept the gender parameter and call it with the previously captured variables to process the specific video file.



In [ ]:
def run_pipeline(video_file, target_language='hi', gender='female'):
    """
    Updated pipeline to handle gender selection and specific video processing.
    """
    if not os.path.exists(video_file):
        print(f'Error: File {video_file} not found.')
        return

    # 1. Extract Audio
    audio = extract_audio(video_file)

    # 2. Speech to Text
    text = speech_to_text(audio)

    # 3. Translate Text
    translated = translate_text(text, target_language)

    # 4. Text to Speech with Gender support
    tts_audio = text_to_speech(translated, target_language, gender)

    # 5. Merge Audio and Video
    merge_audio_video(video_file, tts_audio)

# Call the pipeline with variables from the menu system
specific_video = 'WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4'
print(f'\nStarting pipeline for: {specific_video}')
run_pipeline(specific_video, chosen_lang, chosen_gender)


Starting pipeline for: WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4
Extracting audio from WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4...
MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.
Converting speech to text...


Recognized Text: do well in your life and you've achieved every material game that you want which you should please don't give up on that don't become a philosopher before you become rich
Translating text to te...
Translated Text: మీ జీవితంలో బాగా చేయండి మరియు మీరు కోరుకునే ప్రతి మెటీరియల్ గేమ్‌ను మీరు సాధించారు, దయచేసి మీరు దానిని వదులుకోవద్దు, మీరు ధనవంతులు కావడానికి ముందు తత్వవేత్తగా మారకండి
Generating translated speech in te...
Selected Gender Logic: male (ID: 1)
Merging audio with video...
Moviepy - Building video translated_video.mp4.
MoviePy - Writing audio in translated_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video translated_video.mp4



t:  99%|█████████▉| 263/265 [00:09<00:00, 24.57it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4, 2764800 bytes wanted but 0 bytes read,at frame 262/265, at time 8.73/8.82 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready translated_video.mp4
Done! Output: translated_video.mp4


**Reasoning**:
I will update the run_pipeline function to accept the gender parameter and call it with the previously captured variables to process the specific video file.



In [ ]:
def run_pipeline(video_file, target_language='hi', gender='female'):
    """
    Updated pipeline to handle gender selection and specific video processing.
    """
    if not os.path.exists(video_file):
        print(f'Error: File {video_file} not found.')
        return

    # 1. Extract Audio
    audio = extract_audio(video_file)

    # 2. Speech to Text
    text = speech_to_text(audio)

    # 3. Translate Text
    translated = translate_text(text, target_language)

    # 4. Text to Speech with Gender support
    tts_audio = text_to_speech(translated, target_language, gender)

    # 5. Merge Audio and Video
    merge_audio_video(video_file, tts_audio)

# Call the pipeline with variables from the menu system
specific_video = 'WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4'
print(f'\nStarting pipeline for: {specific_video}')
run_pipeline(specific_video, chosen_lang, chosen_gender)


Starting pipeline for: WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4
Extracting audio from WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4...
MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.
Converting speech to text...
Recognized Text: do well in your life and you've achieved every material game that you want which you should please don't give up on that don't become a philosopher before you become rich
Translating text to te...
Translated Text: మీ జీవితంలో బాగా చేయండి మరియు మీరు కోరుకునే ప్రతి మెటీరియల్ గేమ్‌ను మీరు సాధించారు, దయచేసి మీరు దానిని వదులుకోవద్దు, మీరు ధనవంతులు కావడానికి ముందు తత్వవేత్తగా మారకండి
Generating translated speech in te...
Selected Gender Logic: male (ID: 1)
Merging audio with video...
Moviepy - Building video translated_video.mp4.
MoviePy - Writing audio in translated_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video translated_video.mp4



t:  98%|█████████▊| 261/265 [00:09<00:00, 15.38it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4, 2764800 bytes wanted but 0 bytes read,at frame 262/265, at time 8.73/8.82 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t: 100%|█████████▉| 264/265 [00:09<00:00, 14.85it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4, 2764800 bytes wanted but 0 bytes read,at frame 263/265, at time 8.77/8.82 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready translated_video.mp4
Done! Output: translated_video.mp4


# Task
Update the video translation pipeline to support gender selection and a numeric menu-based input system. Modify the `text_to_speech` function to accept a `gender` parameter and include logic to handle choices between male and female voices. Implement a numeric menu using `input()` that allows selecting a gender (1: Men, 2: Women) and a target language (1: Telugu, 2: Hindi, 3: English, 4: Tamil, 5: Spanish, 6: Kannada). Finally, refactor the `run_pipeline` call to use these selections to process the video "WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4" and generate the final output file "translated_video.mp4".

## Configure TTS for Gender Handling

### Subtask:
Modify the `text_to_speech` function to accept a `gender` parameter and include logic to handle choices between male and female voices.


**Reasoning**:
I will redefine the text_to_speech function to include the gender parameter and the requested logic as specified in the subtask instructions.



In [ ]:
def text_to_speech(text, lang='hi', gender='female', output_audio='translated_audio.mp3'):
    """
    Generates translated speech with a specified language and gender preference.
    """
    print(f'Generating translated speech in {lang}...')

    # Logic to handle/acknowledge the selected gender for simulation
    # Note: gTTS typically uses a single default voice per language.
    print(f'Selected Voice Gender: {gender}')

    # Language mapping logic
    lang_map = {'telugu': 'te', 'hindi': 'hi', 'spanish': 'es', 'french': 'fr', 'tamil': 'ta'}
    lang_code = lang_map.get(lang.lower(), lang)

    # Synthesis using gTTS
    tts = gTTS(text=text, lang=lang_code)
    tts.save(output_audio)

    return output_audio

print('Function text_to_speech updated successfully.')

Function text_to_speech updated successfully.


## Execute Numeric Selection Menu

### Subtask:
Implement and run a numeric menu system to capture user preferences for gender and target language.


**Reasoning**:
I will implement a numeric menu-based input system using `input()` to capture user preferences for gender and target language, and then map those selections to the corresponding internal values.



In [ ]:
def get_numeric_preferences():
    # 1. Gender Selection Menu
    print("\n--- Select Gender ---")
    print("1: Men (male)")
    print("2: Women (female)")
    gender_choice = input("Enter your choice (1 or 2): ")

    # 2. Language Selection Menu
    print("\n--- Select Target Language ---")
    print("1: Telugu (te)")
    print("2: Hindi (hi)")
    print("3: English (en)")
    print("4: Tamil (ta)")
    print("5: Spanish (es)")
    print("6: Kannada (kn)")
    lang_choice = input("Enter your choice (1-6): ")

    # 3. Mapping Logic
    gender_map = {'1': 'male', '2': 'female'}
    selected_gender = gender_map.get(gender_choice, 'female') # Default to female

    lang_map = {
        '1': 'te',
        '2': 'hi',
        '3': 'en',
        '4': 'ta',
        '5': 'es',
        '6': 'kn'
    }
    selected_lang = lang_map.get(lang_choice, 'hi') # Default to Hindi

    # 4. Print resolved selections
    print(f"\nResolved Selections:")
    print(f"Selected Gender: {selected_gender}")
    print(f"Selected Language Code: {selected_lang}")

    return selected_lang, selected_gender

# Execute to capture input for the next steps
chosen_lang, chosen_gender = get_numeric_preferences()


--- Select Gender ---
1: Men (male)
2: Women (female)
Enter your choice (1 or 2): 1

--- Select Target Language ---
1: Telugu (te)
2: Hindi (hi)
3: English (en)
4: Tamil (ta)
5: Spanish (es)
6: Kannada (kn)
Enter your choice (1-6): 1

Resolved Selections:
Selected Gender: male
Selected Language Code: te


## Process Video Pipeline

### Subtask:
Re-run the `run_pipeline` function using your selections to process 'WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4' and generate the 'translated_video.mp4'.


**Reasoning**:
I will re-run the run_pipeline function using the variables captured from the numeric menu to process the specified video file and generate the translated output.



In [ ]:
def run_pipeline(video_file, target_language='hi', gender='female'):
    """
    Executes the translation pipeline using specific video, language, and gender parameters.
    """
    if not os.path.exists(video_file):
        print(f'Error: File {video_file} not found.')
        return

    # 1. Extract Audio
    audio = extract_audio(video_file)

    # 2. Speech to Text
    text = speech_to_text(audio)

    # 3. Translate Text
    translated = translate_text(text, target_language)

    # 4. Text to Speech with the specific gender parameter
    tts_audio = text_to_speech(translated, target_language, gender)

    # 5. Merge Audio and Video
    merge_audio_video(video_file, tts_audio, output_video='translated_video.mp4')

# Call the pipeline with the specific video and the variables from the menu selection
specific_video = 'WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4'
print(f'\nStarting pipeline for: {specific_video}')
run_pipeline(specific_video, chosen_lang, chosen_gender)
print('\nPipeline execution complete. File "translated_video.mp4" should be generated.')


Starting pipeline for: WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4
Extracting audio from WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4...
MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.
Converting speech to text...
Recognized Text: do well in your life and you've achieved every material game that you want which you should please don't give up on that don't become a philosopher before you become rich
Translating text to te...
Translated Text: మీ జీవితంలో బాగా చేయండి మరియు మీరు కోరుకునే ప్రతి మెటీరియల్ గేమ్‌ను మీరు సాధించారు, దయచేసి మీరు దానిని వదులుకోవద్దు, మీరు ధనవంతులు కావడానికి ముందు తత్వవేత్తగా మారకండి
Generating translated speech in te...
Selected Voice Gender: male
Merging audio with video...
Moviepy - Building video translated_video.mp4.
MoviePy - Writing audio in translated_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video translated_video.mp4



t:  99%|█████████▉| 263/265 [00:09<00:00, 15.95it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4, 2764800 bytes wanted but 0 bytes read,at frame 262/265, at time 8.73/8.82 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

t: 100%|██████████| 265/265 [00:09<00:00, 15.77it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4, 2764800 bytes wanted but 0 bytes read,at frame 264/265, at time 8.80/8.82 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready translated_video.mp4
Done! Output: translated_video.mp4

Pipeline execution complete. File "translated_video.mp4" should be generated.


**Reasoning**:
The previous code block successfully implemented the refined run_pipeline function and executed the translation process. I will now add this block to the notebook as the final step of the subtask to process the video with the user-selected preferences.



In [ ]:
def run_pipeline(video_file, target_language='hi', gender='female'):
    """
    Executes the translation pipeline using specific video, language, and gender parameters.
    """
    if not os.path.exists(video_file):
        print(f'Error: File {video_file} not found.')
        return

    # 1. Extract Audio
    audio = extract_audio(video_file)

    # 2. Speech to Text
    text = speech_to_text(audio)

    # 3. Translate Text
    translated = translate_text(text, target_language)

    # 4. Text to Speech with the specific gender parameter
    tts_audio = text_to_speech(translated, target_language, gender)

    # 5. Merge Audio and Video
    merge_audio_video(video_file, tts_audio, output_video='translated_video.mp4')

# Call the pipeline with the specific video and the variables from the menu selection
specific_video = 'WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4'
print(f'\nStarting pipeline for: {specific_video}')
run_pipeline(specific_video, chosen_lang, chosen_gender)
print('\nPipeline execution complete. File "translated_video.mp4" should be generated.')


Starting pipeline for: WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4
Extracting audio from WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4...
MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.
Converting speech to text...
Recognized Text: do well in your life and you've achieved every material game that you want which you should please don't give up on that don't become a philosopher before you become rich
Translating text to te...
Translated Text: మీ జీవితంలో బాగా చేయండి మరియు మీరు కోరుకునే ప్రతి మెటీరియల్ గేమ్‌ను మీరు సాధించారు, దయచేసి మీరు దానిని వదులుకోవద్దు, మీరు ధనవంతులు కావడానికి ముందు తత్వవేత్తగా మారకండి
Generating translated speech in te...
Selected Voice Gender: male
Merging audio with video...
Moviepy - Building video translated_video.mp4.
MoviePy - Writing audio in translated_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video translated_video.mp4



t:  99%|█████████▉| 263/265 [00:09<00:00, 25.74it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4, 2764800 bytes wanted but 0 bytes read,at frame 262/265, at time 8.73/8.82 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready translated_video.mp4
Done! Output: translated_video.mp4

Pipeline execution complete. File "translated_video.mp4" should be generated.


**Reasoning**:
The previous code block successfully implemented the refined run_pipeline function and executed the translation process. I will now add this block to the notebook as the final step of the subtask to process the video with the user-selected preferences.



In [ ]:
def run_pipeline(video_file, target_language='hi', gender='female'):
    """
    Executes the translation pipeline using specific video, language, and gender parameters.
    """
    if not os.path.exists(video_file):
        print(f'Error: File {video_file} not found.')
        return

    # 1. Extract Audio
    audio = extract_audio(video_file)

    # 2. Speech to Text
    text = speech_to_text(audio)

    # 3. Translate Text
    translated = translate_text(text, target_language)

    # 4. Text to Speech with the specific gender parameter
    tts_audio = text_to_speech(translated, target_language, gender)

    # 5. Merge Audio and Video
    merge_audio_video(video_file, tts_audio, output_video='translated_video.mp4')

# Call the pipeline with the specific video and the variables from the menu selection
specific_video = 'WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4'
print(f'\nStarting pipeline for: {specific_video}')
run_pipeline(specific_video, chosen_lang, chosen_gender)
print('\nPipeline execution complete. File "translated_video.mp4" should be generated.')


Starting pipeline for: WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4
Extracting audio from WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4...
MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.
Converting speech to text...


Recognized Text: do well in your life and you've achieved every material game that you want which you should please don't give up on that don't become a philosopher before you become rich
Translating text to te...
Translated Text: మీ జీవితంలో బాగా చేయండి మరియు మీరు కోరుకునే ప్రతి మెటీరియల్ గేమ్‌ను మీరు సాధించారు, దయచేసి మీరు దానిని వదులుకోవద్దు, మీరు ధనవంతులు కావడానికి ముందు తత్వవేత్తగా మారకండి
Generating translated speech in te...
Selected Voice Gender: male
Merging audio with video...
Moviepy - Building video translated_video.mp4.
MoviePy - Writing audio in translated_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video translated_video.mp4



t:  98%|█████████▊| 260/265 [00:10<00:00, 22.98it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4, 2764800 bytes wanted but 0 bytes read,at frame 262/265, at time 8.73/8.82 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t: 100%|█████████▉| 264/265 [00:10<00:00, 23.42it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file WhatsApp Video 2026-02-18 at 9.18.28 PM.mp4, 2764800 bytes wanted but 0 bytes read,at frame 263/265, at time 8.77/8.82 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready translated_video.mp4
Done! Output: translated_video.mp4

Pipeline execution complete. File "translated_video.mp4" should be generated.


# Task
Download the final translated video "translated_video.mp4" to your local machine using `google.colab.files.download`.

## Download Translated Video

### Subtask:
Prompt the user to download the final translated video file from the Colab environment to their local machine.


**Reasoning**:
I will add a code cell to initiate the download of the 'translated_video.mp4' file from the Google Colab environment using the files.download() method.



In [ ]:
from google.colab import files

# Initiate the download of the translated video
print('Downloading translated_video.mp4...')
files.download('translated_video.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Final Task

### Subtask:
Confirm that the download process for 'translated_video.mp4' has been initiated and verify the completion of the task.


## Summary:

### Q&A
**How was the translated video transferred to the local machine?**
The file `translated_video.mp4` was transferred by using the `google.colab.files.download` function, which triggers a native browser download prompt from the Colab environment.

### Data Analysis Key Findings
* **Module Utilization**: The process successfully utilized the `google.colab` library to bridge the gap between the cloud-based file system and the local environment.
* **Execution Verification**: The system output confirmed that the download process for `translated_video.mp4` was initiated successfully following the completion of any prior video processing steps.
* **User Interface Interaction**: The script effectively generated the necessary JavaScript triggers required for the browser to handle the file transfer.

### Insights or Next Steps
* **Verification**: Ensure the downloaded file size matches the processed version on the server to confirm no data corruption occurred during the transfer.
* **Automation**: For future workflows involving multiple files, consider zipping assets before calling the download function to streamline the local saving process.
